# Raw data processing template
- This code can be applied for other .tsv files, with necessary changes described in code
- **IMPORTANT**: this script works only with Eurostat `.tsv` files
- the result of this script is a `.csv` file

- Change according to file

In [1]:
import pandas as pd
import numpy

url = "../../Data/immigration/immigration_citizenship.tsv"
table_raw = pd.read_csv(url,sep='\t')

#Edit year range to corresponding in .tsv file
year_range = range(1998,2024,1)

file_save_path = "../../Data/immigration/immigration_citizenship.csv"


- Important changes here:

In [2]:
# CHANGE HERE 
#create corresponding columns from first column
age = []
agedef = []
birth_country = []
sex = []
country = []

for index,row in table_raw.iterrows():
    title_split = (row[0].split(","))
    
    # add data to respective columns
    age_str = title_split[1][1:]
    
    if age_str.isnumeric():
        age.append(age_str)

    else:
        if title_split[1]=="Y_GE100":
            age.append(str(100))
        elif title_split[1]=="Y_LT1":
            age.append(str(0))
        else:
            age.append(title_split[1]) 

    agedef.append(title_split[2])
    birth_country.append(title_split[3])
    sex.append(title_split[5])
    country.append(title_split[6])
    
    
# add lists into table
table_raw["age"]= age
table_raw["agedef"] = agedef
table_raw["birth_country"] = birth_country
table_raw["sex"]=sex
table_raw["country"]=country

#drop the first column 
# TODO this code could be optimised by providing index, please do so if you can
table_raw.drop(columns=["freq,age,agedef,citizen,unit,sex,geo\TIME_PERIOD"],inplace=True)


<>:41: SyntaxWarning: invalid escape sequence '\T'
<>:41: SyntaxWarning: invalid escape sequence '\T'
/tmp/ipykernel_64540/2323186131.py:41: SyntaxWarning: invalid escape sequence '\T'
  table_raw.drop(columns=["freq,age,agedef,citizen,unit,sex,geo\TIME_PERIOD"],inplace=True)
/tmp/ipykernel_64540/2323186131.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  title_split = (row[0].split(","))


#### Reindexing, renaming columns, adding ISO3 codes


In [3]:
# import ISO3 codes 
country_codes = pd.read_csv("../../Data/country-region-codes.csv")

#Creating new column names for renaming
new_columns = {}
for column in table_raw.columns:
    if column.strip().isnumeric():
        new_columns[column] = int(column.strip())
    else:
        new_columns[column]=column
        
table = table_raw.rename(columns=new_columns)
table.rename(columns={"country":"alpha-2"},inplace=True) #Renaming country column to alpha-2 so it matches with country-region-codes

# Merging table on country codes. 
table = pd.merge(table,country_codes,on="alpha-2").drop(columns=["country-code","alpha-2"]).rename(columns={"name":"Country","alpha-3":"ISO3"})

#reordering
new_order = list(table.columns[-6:])+[i for i in year_range]# IMPORTANT to set -6 to number of non numeric columns

table = table[new_order]

#### Fixing values
Renaming : to NaN
fixing provisional data

In [4]:
replaced = table.copy()

#fix numeric values
for column in replaced.columns:
    replaced[column] = replaced[column].apply(str.strip)

replaced.replace(to_replace=":",value=numpy.nan,inplace=True)

#tags
tags = ["b","p","ep","e","be","bep","P"]

#I used Deepseek AI for this, it was quite complex
pattern = r'\b(?:' + '|'.join(tags) + r')\b'

# Apply the replacement to all columns (or specify specific columns)
df = replaced.apply(lambda x: x.str.replace(pattern, '', regex=True).str.strip())



#### Saving to file
- Uncomment to save file

In [5]:
display(df.head())

#Uncomment to save file
# df.to_csv(file_save_path,index=False)

,age,agedef,birth_country,sex,Country,ISO3,1998,1999,2000,2001,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,TOTAL,COMPLET,FOR_STLS,F,Austria,AUT,28638,35797,NaN,34825,...,48788,65441,55097,48739,45444,46329,42715,46214,97378,64503
1,TOTAL,COMPLET,FOR_STLS,F,Belgium,BEL,29382,33021,31296,39605,...,51236,56379,50222,51820,55655,59833,47172,55678,93814,71027
2,TOTAL,COMPLET,FOR_STLS,F,Bulgaria,BGR,NaN,NaN,NaN,NaN,...,7295,6276,5522,5844,5960,6588,6190,6852,8357,18085
3,TOTAL,COMPLET,FOR_STLS,F,Switzerland,CHE,36508,NaN,42245,NaN,...,62616,61336,59968,57238,57673,57407,53380,57705,66610,117356
4,TOTAL,COMPLET,FOR_STLS,F,Cyprus,CYP,NaN,NaN,NaN,NaN,...,5630,7362,7437,9498,10230,11953,12580,11222,15632,19850
